In [9]:

from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [12]:
os.path.join('immagini\\immagini_originali', 'Ale.jpg')

'immagini\\immagini_originali\\Ale.jpg'

In [18]:
# Carico il modello vgg19 pre-addestrato

model = models.vgg19(weights = True).features

In [19]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        
        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained = True).features[:29]
        
    def forward(self, x):
        features = []
        
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            
            if str(layer_num) in self.chosen_features:
                features.append(x)
                
        return features

In [20]:
# Qui carico le immagini di cui ho bisogno
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)

device = torch.device("cpu")
image_size = 256

loader = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

# carico immagine originale
path_img_orig = "C:\\Users\\aless\\Documents\\03_git_hub_personale\\01_transfer_style\\immagini\\immagini_originali\\Ale.jpeg"
original_img = load_image(path_img_orig)
# carico immagine con lo stile richiesto
path_img_style = "C:\\Users\\aless\\Documents\\03_git_hub_personale\\01_transfer_style\\immagini\\immagini_stile\\sentiero.jfif"
style_img = load_image(path_img_style)


model = VGG().to(device).eval()

#generated = torch.randn(original_img.shape, device = device, requires_grad = True)
generated = original_img.clone().requires_grad_(True)

In [17]:
# hyperparameters

total_steps = 1000
learning_rate = 0.05
alpha = 1
beta = 0.001

optimizer = optim.Adam([generated], lr = learning_rate)

for step in range(total_steps):
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)
    
    style_loss = original_loss = 0
    
    for gen_feature, orig_feature, style_feature in zip(generated_features, original_img_features, style_features):
        batch_size, channel, height, width = gen_feature.shape
        original_loss = torch.mean((gen_feature - orig_feature) **2)
        
        #compute the Gram Matrix
        G = gen_feature.view(channel, height*width).mm(
            gen_feature.view(channel, height*width).t()
        )
        
        A = style_feature.view(channel, height*width).mm(
            style_feature.view(channel, height*width).t()
        )
        
        style_loss += torch.mean((G-A)**2)
        
        
    total_loss = alpha*original_loss + beta*style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        generated.save("generated.png")    # c'è qualche problema, domani va ricontrollato!
        

tensor(453775.8438, grad_fn=<AddBackward0>)


AttributeError: 'Tensor' object has no attribute 'save'